# To comply with cinfidentiality obligations, the code has been simplified to demonstrate fundamental concepts of an AI agent chatbot.

In [1]:
# AMikoXR Chatbot using LangChain, Open AI, RAG, Qdrant, Groq  

### Install Libraries

!pip install -U \
LangChain \
langchain_openai \
OpenAI \
Datasets \
qdrant-client \
Tiktoken

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv('./.env')

print(os.environ)

chat = ChatOpenAI(
    model='gpt-3.5-turbo',
    openai_api_key=''
)

from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)


messages = [
    SystemMessage(content="You are a helful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I would like to understand machine learnin.")
]

res= chat.invoke(messages)
res

print(res.content)

messages.append(res)

prompt = HumanMessage(
    content="Whats the difference between supervised and unsupervised"
)

messages.append(prompt)

res= chat.invoke(messages)
print(res.content)

messages.append(res)

prompt = HumanMessage(
    content="What is so special about Mistral 7B?"
)
messages.append(prompt)
res = chat.invoke(messages)

print(res.content)

llmchain_information= [
    "LLMChain is an AI model that combines the capabilities of Large Language Models (LLMs) with blockchain technology. It's a framework that enables the creation of decentralized, transparent, and secure AI applications."
]

source_knowledge = "\n".join(llmchain_information) 

query = "Can you tell me about LLMChain in LangChain?"

augmented_prompt = f"""Using the contexts below to answer the question.

Contexts:
{source_knowledge}

Question: {query}"""

prompt = HumanMessage(
    content=augmented_prompt
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

from datasets import load_dataset

dataset = load_dataset("infoslack/mistral-7b-arxiv-paper-chunked", split="train")

dataset

dataset[0]

data = dataset.to_pandas()

data.head()

docs = data[['chunk','source']]
docs.head()

from langchain_openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=""
)

texts = [
    'this is one chunk',
    'this is the second chunk of text'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

from datasets import load_dataset

dataset = load_dataset("infoslack/mistral-7b-arxiv-paper-chunked", split="train")

dataset

dataset[0]

data = dataset.to_pandas()

data

docs = data[['chunk','source']]
docs.head()

#RAG
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(docs, page_content_column="chunk")
documents = loader.load()

documents[0]

documents[0].metadata

# Set the environment variables
os.environ['QDRANT_URL'] = ''
os.environ['QDRANT_API_KEY'] = ''

print("Qdrant URL:", os.getenv('QDRANT_URL'))
print("API Key:", os.getenv('QDRANT_API_KEY'))

from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=""
)

# Get URL and API Key
url = os.getenv('QDRANT_URL')
api_key = os.getenv('QDRANT_API_KEY')

# Debugging output
print("Connecting to Qdrant at:", url)
print("Using API Key:", api_key)

# Ensure URL is valid
if not url:
    print("Error: Qdrant URL is not set.")
    exit(1)

# Attempt to create Qdrant instance
try:
    qdrant = Qdrant.from_documents(
        documents=documents,  # Ensure 'documents' is defined elsewhere
        embedding=embeddings,
        url=url,
        collection_name='chatbot',
        api_key=api_key
    )
except Exception as e:
    print("Failed to connect to Qdrant:", str(e))

query = "What is so special about Mistral 7B?"
qdrant.similarity_search(query, k=3)

def custom_prompt(query: str):
    results = qdrant.similarity_search(query, k=3)
    source_knowledge = '\n'.join([x.page_content for x in results])
    augment_prompt = f"""Using the contexts below, answer the query:
    
    Contexts:
    {source_knowledge}
    
    Query: {query}"""
    return augment_prompt

print(custom_prompt(query))

prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

#Groq
!pip install langchain-groq

from langchain_groq import ChatGroq
chat = ChatGroq(temperature=0, model_name='mixtral-8x7b-32768', groq_api_key='')

prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)



  Obtaining dependency information for LangChain from https://files.pythonhosted.org/packages/0d/29/635343c0d155997569b544d26da5a2a9ebade2423baffc9cd6066b01a386/langchain-0.2.16-py3-none-any.whl.metadata
  Obtaining dependency information for OpenAI from https://files.pythonhosted.org/packages/99/ae/e8fb328fc0fc20ae935950b1f7160de8e2631a5997c2398c9b8a8cc502f8/openai-1.44.0-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.3.0,>=0.2.38 from https://files.pythonhosted.org/packages/1c/e4/501fbe904530dad6ed80f03b188d7602081560dd5cc0bcf0b3c51778c314/langchain_core-0.2.38-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------ --------------------------------- 0.2/1.0 MB 4.6 MB/s eta 0:00:01
   ---------------------- ----------------- 0.6/1.0 MB 5.0 MB/s eta 0:00:01
   -------------------------- ------------- 0.7/1.0 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 0.7/1.0 MB 4.2 MB/s eta 0:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymemgpt 0.3.22 requires protobuf==3.20.0, but you have protobuf 5.27.3 which is incompatible.
pymemgpt 0.3.22 requires tiktoken<0.6.0,>=0.5.1, but you have tiktoken 0.7.0 which is incompatible.


environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\Kasra\\AppData\\Roaming', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'DESKTOP-KVL4TLC', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'EFC_7216': '1', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\Kasra', 'LOCALAPPDATA': 'C:\\Users\\Kasra\\AppData\\Local', 'LOGONSERVER': '\\\\DESKTOP-KVL4TLC', 'NUMBER_OF_PROCESSORS': '4', 'ONEDRIVE': 'C:\\Users\\Kasra\\OneDrive - stevens.edu', 'ONEDRIVECOMMERCIAL': 'C:\\Users\\Kasra\\OneDrive - stevens.edu', 'ONEDRIVECONSUMER': 'C:\\Users\\Kasra\\OneDrive', 'ONLINESERVICES': 'Online Services', 'OS': 'Windows_NT', 'PATH': 'C:\\Users\\Kasra\\anaconda3;C:\\Users\\Kasra\\anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\Kasra\\anaconda3\\Library\\usr\\bin;C:\\Users\\Kasra\\anaconda3\\Li

Qdrant URL: https://b5512ba6-9796-4afa-974d-62bda96948c3.europe-west3-0.gcp.cloud.qdrant.io/
API Key: b2aZn45Vr3_Olldq5xk7iAPmaoRzvifPb0DHayKBGZTKIoWlQS2k9Q
Connecting to Qdrant at: https://b5512ba6-9796-4afa-974d-62bda96948c3.europe-west3-0.gcp.cloud.qdrant.io/
Using API Key: b2aZn45Vr3_Olldq5xk7iAPmaoRzvifPb0DHayKBGZTKIoWlQS2k9Q
Using the contexts below, answer the query:
    
    Contexts:
    Mistral 7B
Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,
Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,
Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,
Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,
William El Sayed
Abstract
We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, ma

Mistral 7B is a 7-billion parameter language model that is engineered for superior performance and efficiency. It outperforms the best open 13B model (Llama 2) across all evaluated benchmarks and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation. Mistral 7B uses grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to handle sequences of arbitrary length with reduced inference cost. It also provides a model fine-tuned to follow instructions, Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model on both human and automated benchmarks. Mistral 7B is released under the Apache 2.0 license and is easy to deploy and integrate with cloud platforms and Hugging Face. It is also designed for ease of fine-tuning across a wide range of tasks. Overall, Mistral 7B takes a significant step in balancing the goals of getting high performance while keeping large language models efficient.
